In [ ]:
### This is a vignette on how to use the singletCode package

In [1]:
# Install singletCode package

!pip3 install singletCode

from singletCode import check_sample_sheet, get_singlets


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Read in input sheet

import pandas as pd
path = "/Volumes/fsmresfiles/Basic_Sciences/CDB/GoyalLab/People/MadelineMelzer/ZhangMelzerEtAl/data/vignetteTests/singletCodePackageVignetteData"
pathToInputSheet = f"{path}/inputFiles/JiangEtAlSubset_InputSheet.csv"
df = pd.read_csv(pathToInputSheet)

In [4]:
# Check formatting of input sheet

check_sample_sheet(df)

The sample sheet provided can be used as input to get_singlets to get a list of singlets identified.


In [5]:
# Identify singlets from input sheet

outputPath = f"{path}/outputFiles/"
cellLabelList, stats = get_singlets(df, dataset_name= "JiangEtAlSubset", save_all_singlet_categories = True, output_path=outputPath)

#Saving the stats and the singlet list

stats.to_csv(f"{outputPath}/JiangEtAlSubset_stats.csv")
cellLabelList[cellLabelList['label'] == "Singlet"].to_csv(f"{outputPath}/JiangEtAlSubset_singletList.csv")

INFO: Raw data counts: 
sample
1    1306
Name: count, dtype: int64
Total cells for sample 1: 39
INFO: Using ratio based filtering.
Current Sample Adjusted UMI cutoff: 2


100%|██████████| 122/122 [00:00<00:00, 33188.81it/s]

All singlets identified with multiple barcodes are unique? True
Total Singlets: 10
Total Multiplets: 9


In [7]:
#Understanding the output files

import pandas as pd
df[df['cellID'] == 'TGTAAGCGTCTCGCGA'].groupby(['cellID', 'barcode', 'sample']).size().reset_index(name='count').sort_values('count', ascending=False).reset_index(drop=True)

,cellID,barcode,sample,count
0,TGTAAGCGTCTCGCGA,ATTGTTGTTGCAGATGCAGTTGATGCTGATGAAGTTGTACAAGGTC...,1,99
1,TGTAAGCGTCTCGCGA,ATTCGACTTGATCTTCTAGAACATGGTGAACTAGCAGGTGCTGATC...,1,7
2,TGTAAGCGTCTCGCGA,ATACTAGCTCAAGCAGTACTACTACTTCGTCTTCATGCAGAACAAC...,1,6
3,TGTAAGCGTCTCGCGA,ATAGATGCACTTGGTGGTCGAGTTCTAGTTGTAGCTGATCGTCCAG...,1,6
4,TGTAAGCGTCTCGCGA,ATTCGACCAGAACCACATGCAGTTCAACGTGTTCGAGGTGTAGATG...,1,6
...,...,...,...,...
82,TGTAAGCGTCTCGCGA,ATAGTAGTAGCTGTTGGTGTTGAAGTACTTCCTCTTGCTCCTCGTG...,1,1
83,TGTAAGCGTCTCGCGA,ATAGTAGATGAACGTCCTCTACATGTTCTTCGTCAAGTACCAGCAC...,1,1
84,TGTAAGCGTCTCGCGA,ATAGTACATGGTGGACCTGGACTTCGAGATGGAGCTCTTGTTCCTG...,1,1
85,TGTAAGCGTCTCGCGA,ATAGGAGTAGTTGGTGATGGTCTACCAGAAGGTGAAGGTGGAGAAG...,1,1


In [8]:
(df[df['cellID'] == 'AGGCTGCTCTTTCCGG']
 .groupby(['cellID', 'barcode', 'sample'])
 .size()
 .reset_index(name='count')
 .sort_values('count', ascending=False)
 .query('count >= 2')
 .reset_index(drop=True)
)

,cellID,barcode,sample,count
0,AGGCTGCTCTTTCCGG,ATAGGAGTAGTTGGTGATGGTCTACCAGAAGGTGAAGGTGGAGAAG...,1,13
1,AGGCTGCTCTTTCCGG,ATTGAACGTGGAGTTGAACTTGTACTACGAGTACGTCTAGAACATG...,1,2


In [9]:
(df[df['cellID'] == 'GAGGGATGTAACATCC']
 .groupby(['cellID', 'barcode', 'sample'])
 .size()
 .reset_index(name='count')
 .sort_values('count', ascending=False)
 .query('count >= 2')
 .reset_index(drop=True)
)

,cellID,barcode,sample,count
0,GAGGGATGTAACATCC,ATAGGAGTAGTTGGTGATGGTCTACCAGAAGGTGAAGGTGGAGAAG...,1,12
1,GAGGGATGTAACATCC,ATTGAACGTGGAGTTGAACTTGTACTACGAGTACGTCTAGAACATG...,1,2
